In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 42

In [33]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [30]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', gamma= 100, C=0.1)
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.1, gamma=100)

In [31]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.5317371937639198
ROC-AUC score : 0.5


In [32]:
ut.write(model)